<a href="https://colab.research.google.com/github/JoaoBruno09/ai-course-rag-project/blob/master/RAG_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Project -> Financial Analysis

#### THE RAG FLOW:
  1. Retrieve: Get relevant documents (this case pdf documents)
  2. Format: Combine documents into context string
  3. Prompt: Create structured instruction for LLM
  4. Chain: prompt -> llm -> output parser
  5. Generation: LLM produces natural language answer
  6. Return: User gets readable answer

  ---

#### THIS RAG FEATURES:
- Injestion function
- Inference function
- Similarity Search with separators
- Chroma persistence
- LCEL (LangChain Expression Language) chains
- Prompt templates
- LLM integration
- Gradio ChatInterface
- Auto-detection year or report quarterly period using LLM classification model
- One-at-a-time injestion
- Text preprocessing
- Metadata filtering

Vector Database used: https://www.trychroma.com

## Install Dependencies

In [ ]:
%pip install "langchain==0.3.27" -qqq
%pip install "langchain-community==0.3.31" -qqq
%pip install "langchain-openai==0.3.35" -qqq
%pip install "langchain-chroma==0.2.6" -qqq
%pip install pypdf -qqq
%pip install gradio -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Configuration

In [ ]:
import os
from google.colab import userdata

# OpenAI API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["CHROMA_API_KEY"] = userdata.get("CHROMA_API_KEY")
os.environ["CHROMA_TENANT"] = userdata.get("CHROMA_TENANT")
os.environ["DB_NAME"] = userdata.get("DB_NAME")

# Version Management
REPORTS_VERSION = "v1"

# Vector Databases collection name
REPORTS_COLLECTION_NAME = f"googl_reports_{REPORTS_VERSION}"


## Global Variables

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma

# Embeddings Model
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

# LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0 # Deterministic for consistent classification
)

# Model for report year classififcation
classification_llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0  # Deterministic for consistent classification
)

# Vector Databases
reports_vectorstore = Chroma(
  embedding_function=embeddings,
  collection_name=REPORTS_COLLECTION_NAME,  # Version-based naming
  chroma_cloud_api_key=os.getenv("CHROMA_API_KEY"),
  tenant=os.getenv("CHROMA_TENANT"),
  database=os.getenv("DB_NAME")
)

## Injestion Process

In [ ]:
#@title Clean text function

import re

def improvePage(text: str) -> str:
  """
    Method that applies text preprocessing
    - Remove hyphenated line breaks
    - Remove multiple whitespaces
    - Normalize punctuation spacing
    - Remove standalone page numbers
    - Remove whitespace from both the beginning and the end of the string

    Args:
      documentPath (str): path to Document

    Returns:
      str: Processed text
  """

  # Remove hyphenated line breaks
  text = re.sub(r'(\w)-\s+(\w)', r'\1\2', text)

  # Remove multiple whitespaces -> \s+ matches one or more whitespace characters (spaces, tabs, newlines) and replaces them with a single space " "
  text = re.sub(r'\s+', ' ', text)

  # Normalize punctuation spacing
  text = re.sub(r'\s+([.,!?;:])', r'\1', text)
  text = re.sub(r'([.,!?;:])([^\s])', r'\1 \2', text)

  # Remove standalone page numbers -> remove a trailing number only if it's the final token
  text = re.sub(r'(?<=\.)\s*\d+\s*$', '', text)

  # Removes whitespace from both the beginning and the end of the string
  return text.strip()

In [ ]:
#@title Document Classification Chain Creation

def detect_document_properties(report):
  """
  Method retuns the document properties classified by LLM
  - Prompt to detect report year and quarterly period
  - Create detection chain and invoke it
  - Returns JSON with detected values

  Args:
    report: Document report loaded by PDF

  Returns:
    detection_result: JSON with detected values
  """

  # Detection Prompt
  detection_template = ChatPromptTemplate.from_template(
    """
    You are a world-class **Financial Document Analyst** and **Stock Market Expert** specializing in reading and interpreting corporate annual and quarterly reports.

    Your task is to extract two specific pieces of information from the following financial report text:

    1. **Fiscal Year** – the year the report covers (e.g., 2023).
    2. **Fiscal Quarter** – the quarter the report covers, if applicable (Q1, Q2, Q3, Q4).
      - If the document is an **annual report**, set Quarter to `"ANNUAL"`.
      - If quarter cannot be determined, set Quarter to `"UNKNOWN"`.

    ---

    ### Step-by-Step Instructions
    1. Read the provided document text carefully.
    2. Identify the fiscal or calendar year the report refers to (e.g., “for the year ended December 31, 2023” → year = 2023, “For the quarterly period ended June 30, 2025” → year = 2025).
    3. Identify the quarter, if mentioned (e.g., “Quarter ended March 31, 2023” → quarter = Q1, “For the quarterly period ended June 30, 2025” → quarter = Q2).
    4. If multiple years or quarters appear, choose the **most recent** one that the report explicitly covers.
    5. Output the results **strictly in JSON format** as follows: {{"year": "<4-digit-year>", "quarter": "<Q1|Q2|Q3|Q4|ANNUAL|UNKNOWN>"}}

    ---

    ### Document Content
    {content}

    ---

    ### Final Answer:
    Return **only** the JSON block — no extra text or commentary.
    """
  )

  # Detection Chain
  detection_chain = detection_template | classification_llm | StrOutputParser()

  # Detection Document Sample
  sample_content = ""
  for doc in report[:1]:  # First page
      sample_content += doc.page_content + "\n\n"

  # Limit to 500 characters to save costs, but also to identify report year, report quarter and ticker
  sample_content = sample_content[:500]

  # Invoke chain
  detection_result = detection_chain.invoke({
      "content": sample_content
  }).strip().lower()

  return detection_result

In [ ]:
#@title Injestion Function

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import re
import json

def ingest_report(documentPath: str):
  """
  Injestion Function - Run Once (or when data changes)

  Injestion Features:
  - Loads pdf reports using PyPDF loader
  - Auto-detect report year and quarterly period if applicable using a classification LLM (gpt-3.5-turbo)
  - Applies text processing
  - Creates chunks using RecursiveCharacterTextSplitter according to separators
  - Adds report year and quarterly period if applicable to each chunk metadata
  - Creates or appends to chroma collection

  Args:
      documentPath (str): path to Document
  """

  print("\n\n")
  print("-" * 80)
  print(f"STARTING ADBE ANNUAL REPORT '{documentPath}' INGESTION PIPELINE - VERSION {REPORTS_VERSION}")
  print("-" * 80)

  #--------------------------------------------------------------------------------
  # STEP 1: LOAD REPORTS
  #--------------------------------------------------------------------------------
  print("\n[1/6] Loading report...")

  loader = PyPDFLoader(documentPath)
  report = loader.load()

  print(f"✓ Loaded {len(report)} pages from file")

  #--------------------------------------------------------------------------------
  # STEP 2: AUTO-DETECT YEAR AND QUARTER (IF APPLICABLE) REPORT
  #--------------------------------------------------------------------------------
  print(f"\n[2/6] Auto-detecting report year and quarterly period...")

  detection_result = detect_document_properties(report).strip()
  detected_year = json.loads(detection_result)["year"]
  detected_quarter = json.loads(detection_result)["quarter"]

  print(f"✓ Financial Report Year auto-detected: '{detected_year}'")
  print(f"✓ Financial Report Quarterly Period auto-detected: '{detected_quarter}'")

  #--------------------------------------------------------------------------------
  # STEP 3: PREPROCESSING TEXT
  #--------------------------------------------------------------------------------

  print(f"\n[3/6] Applying text preprocessing...")

  for doc in report:
    doc.page_content = improvePage(doc.page_content)

  print(f"✓ Cleaned {len(report)} pages")

  #--------------------------------------------------------------------------------
  # STEP 4: CHUNK DOCUMENT
  #--------------------------------------------------------------------------------
  # WHY CHUNK?
  # - LLMs have token limits (context window)
  # - Smaller chunks = more precise retrieval
  # - Balance: too small (lose context) vs too large (lose precision)

  print(f"\n[4/6] Chunking document...")

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=2000,
      chunk_overlap=250, #18% of 2000 chunk size
      separators=["\n\n", "\n", ". ", " ", ""], # how the chunks are separeted
  )

  chunks = text_splitter.split_documents(report)

  print(f"✓ Split into {len(chunks)} chunks")

  #--------------------------------------------------------------------------------
  # STEP 5: ADD METADATA
  #--------------------------------------------------------------------------------
  print(f"\n[5/6] Enriching chunks with metadata...")

  for chunk in chunks:
    # ADD new metadata (doesn't override existing)
    chunk.metadata.update({
        'report_year': detected_year,
        'report_quarter': detected_quarter
    })

  print(f"✓ Metadata enriched for all chunks:")

  #--------------------------------------------------------------------------------
  # STEP 5: CREATE EMBEDDINGS AND STORE IN CHROMA
  #--------------------------------------------------------------------------------

  print(f"\n[6/6] Creating embeddings and storing in Chroma...")

  reports_vectorstore.add_documents(documents=chunks)

  print(f"✓ Embeddings created and stored")

In [ ]:
#@title Run Injestion

ingest_report("/content/NASDAQ_GOOGL_2018.pdf")
ingest_report("/content/NASDAQ_GOOGL_2019.pdf")
ingest_report("/content/NASDAQ_GOOGL_2020.pdf")
ingest_report("/content/NASDAQ_GOOGL_2021.pdf")
ingest_report("/content/NASDAQ_GOOGL_2022.pdf")
ingest_report("/content/NASDAQ_GOOGL_2023.pdf")
ingest_report("/content/NASDAQ_GOOGL_2024.pdf")
ingest_report("/content/NASDAQ_GOOGL_2025_1Q.pdf")
ingest_report("/content/NASDAQ_GOOGL_2025_2Q.pdf")




--------------------------------------------------------------------------------
STARTING ADBE ANNUAL REPORT '/content/NASDAQ_GOOGL_2018.pdf' INGESTION PIPELINE - VERSION v1
--------------------------------------------------------------------------------

[1/6] Loading report...
✓ Loaded 107 pages from file

[2/6] Auto-detecting report year and quarterly period...
✓ Financial Report Year auto-detected: '2017'
✓ Financial Report Quarterly Period auto-detected: 'annual'

[3/6] Applying text preprocessing...
✓ Cleaned 107 pages

[4/6] Chunking document...
✓ Split into 240 chunks

[5/6] Enriching chunks with metadata...
✓ Metadata enriched for all chunks:



--------------------------------------------------------------------------------
STARTING ADBE ANNUAL REPORT '/content/NASDAQ_GOOGL_2019.pdf' INGESTION PIPELINE - VERSION v1
--------------------------------------------------------------------------------

[1/6] Loading report...
✓ Loaded 99 pages from file

[2/6] Auto-detecting repo

## Inference Process

In [ ]:
#@title Query Classification Chain Creation

"""
  Method retuns the query properties classified by LLM
  - Prompt to detect report year and quarterly period
  - Create detection chain and invoke it
  - Returns JSON with detected values

  Args:
    query: User query

  Returns:
    year_quarter_report: JSON with detected values
"""

def detect_query_properties(query):

  # Year or Quarter Detection Prompt
  classification_template = ChatPromptTemplate.from_template(
    """
    You are a precise text classification model specialized in identifying the **reporting period** referenced in a financial question. You must extract the relevant year and quarterly period the user is referring to.

    Your goal is to classify the user's question by identifying each **year** and **quarterly period** mentioned. Classify according to the following categories:

    - **2025** → Refers to a specific **year** (e.g., “2025,” “next year,” “2025 performance”)
    - **1Q** → Refers to the **first quarter** (Q1, January–March, or similar references)
    - **2Q** → Refers to the **second quarter** (Q2, April–June, or similar references)
    - **3Q** → Refers to the **third quarter** (Q3, July–September, or similar references)
    - **4Q** → Refers to the **fourth quarter** (Q4, October–December, or similar references)
    - **ANNUAL** → Refers to a **full-year** or **annual report** (e.g., “annual report,” “full-year,” “yearly performance,” or references to the company’s complete fiscal year)
    - **UNKNOWN** → If the question does not clearly specify any year or quarter.

    ---

    ### Step-by-Step Instructions
    - If there is any **ambiguity** or the period is **not clear**, output **UNKNOWN**.
    - Ensure that the classification is **direct** and **precise** without explanation.
    - Output the results **strictly in JSON format** as follows: {{"user-year": "<4-digit-year>", "user-quarter": "<Q1|Q2|Q3|Q4|ANNUAL|UNKNOWN>"}}

    ---

    ### USER:
    Question: {query}

    ### Classification:
    Return **only** the JSON block — no extra text or commentary.
    """
  )

  # Year or Quarter Detection Chain
  classification_chain = classification_template | classification_llm | StrOutputParser()

  # Invoke chain
  year_quarter_report = classification_chain.invoke({
      "query": query
  }).strip().lower()

  return year_quarter_report

In [ ]:
#@title RAG Chain Creation

"""
  Method retuns the RAG chain
  - Prompt to analyse stock market reports according to user query
  - Create a RAG chain and return it

  Returns:
    chain: created RAG chain
"""

def create_rag_chain():
  prompt_template = ChatPromptTemplate.from_template(
    """
    You are a world-class **Stock Analyst** and **Financial Report Specialist** trained in **fundamental and value investing** following the philosophies of **Warren Buffett**, **Peter Lynch**, and **Benjamin Graham**.

    Your expertise lies in reading **corporate annual, quarterly, and earnings reports** to evaluate a company’s **intrinsic business quality and long-term value**.

    ---

    ### Your Mission
    Based **only** on the reports provided below, you must:

    1. Provide a **professional summary and analysis** of the company.
    2. Describe the company’s **financial performance year by year**, including revenue, earnings, margins, cash flow, debt, and other key metrics.
    3. **Answer the user’s question** clearly and concisely, in the **same language** as the question.
    4. Assign a **Company Evaluation Score (0–100)** according to **Buffett–Lynch–Graham value-investing principles**.

    ---

    ### Evaluation Principles

    Use these guiding principles when scoring:

    **Warren Buffett – Business Quality & Moat**
    - Understandable business model (“circle of competence”)
    - Consistent earnings and return on equity
    - Sustainable competitive advantage (economic moat)
    - Honest, capable management

    **Peter Lynch – Growth at a Reasonable Price (GARP)**
    - Clear growth story grounded in fundamentals
    - Reasonable valuation relative to earnings and assets
    - Healthy financial structure (manageable debt, solid cash flow)

    **Benjamin Graham – Margin of Safety & Intrinsic Value**
    - Stock price meaningfully below intrinsic value
    - Conservative accounting and prudent capital allocation
    - Strong balance sheet and predictable profits

    ---

    ### Scoring Framework

    | Score Range | Interpretation |
    |--------------|----------------|
    | **0–20** | Very weak fundamentals, no margin of safety, deteriorating business |
    | **21–40** | Weak financials or overvaluation, limited safety margin |
    | **41–60** | Average fundamentals, fairly valued, moderate stability |
    | **61–80** | Strong fundamentals, good margin of safety, stable growth |
    | **81–100** | Exceptional quality, durable moat, undervalued or superior long-term compounding potential |

    Consider only what appears in the reports, market data, or company news provided — never speculate beyond that information.

    ---

    ### Compliance Rules
    - **Do NOT** give trading or investment recommendations (no “buy,” “sell,” or “hold”).
    - **Do NOT** use external data or personal opinions.
    - If the question is **not related to the reports**, respond exactly with:
      > "The question is not related to the provided reports."

    ---

    ### Input Variables

    **Reports:**
    {context}

    **Question:**
    {query}

    ---

    ### Output Instructions
    Respond in the **same language** as the question.
    Structure your answer in **four parts**:

    1. **Introduction** – Brief overview of the company, business model, and recent financial context.
    2. **Financial Analysis** – Summarize revenue, net income, operating margins, cash flow, debt levels, and other key metrics. Highlight growth trends, anomalies, or financial strengths/weaknesses. If possible refer the fiscal year that each metric is based on and compare the growth along of the years.
    3. **Qualitative Analysis** – Evaluate the company’s competitive advantage, management quality, and alignment with value investing principles.
    4. **Evaluation Score** – Assign a **Company Evaluation Score (0–100)** based strictly on Buffett–Lynch–Graham criteria.
      > **Evaluation Score:** <numeric_value>

    ---

    ### ✅ Final Answer:
    """
  )

  print("✓ Prompt template created")

  # The pipe (|) operator connects components (output from the last is the input of the next)
  # Read left to right: prompt → llm → parser

  print("\n[5/6] Composing chain...")

  chain = prompt_template | llm | StrOutputParser()

  return chain

In [ ]:
#@title Inference function

def inference(query: str):
  """
  INFERENCE PIPELINE - Run Per User Query

  Inference Features:
  - Auto-detect report year and quarterly period if applicable using a classification LLM (gpt-3.5-turbo)
  - Build metadata filter
  - Similarity search with metadata filter
  - Context formatting for LLM
  - Prompt Engineering for RAG Chain
  - LLM generation response

  Args:
      query (str): User's question

  Returns:
      str: Natural language answer
  """

  print("="*80)
  print(f"RUNNING INFERENCE - VERSION {REPORTS_VERSION}")
  print("="*80)

  #--------------------------------------------------------------------------------
  # STEP 1:  AUTO-DETECT YEAR AND QUARTER (IF APPLICABLE) REPORT
  #--------------------------------------------------------------------------------
  print(f"\n[1/6] Detecting report year...")

  detection_result = detect_query_properties(query)
  user_detected_year = json.loads(detection_result)["user-year"]
  user_detected_quarter = json.loads(detection_result)["user-quarter"]

  print(f"✓ User question year auto-detected: '{user_detected_year}'")
  print(f"✓ User question quarterly period auto-detected: '{user_detected_quarter}'")

  #--------------------------------------------------------------------------------
  # STEP 2: BUILD FILTER
  #--------------------------------------------------------------------------------
  print(f"\n[2/6] Building metadata filter...")

  filter_conditions = {}

  # Add year filter if the detected year is not 'UNKNOWN'
  if user_detected_year.lower() != 'UNKNOWN'.lower():
      filter_conditions['report_year'] = {'$eq': user_detected_year}

  # Add quarter filter if the detected quarter is not 'UNKNOWN'
  if user_detected_quarter.lower() != 'UNKNOWN'.lower():
      filter_conditions['report_quarter'] = {'$eq': user_detected_quarter}

  # Wrap the filter conditions in an '$and' operator to combine them
  if len(filter_conditions) > 1:
    filter_conditions = {'$or': [{key: value} for key, value in filter_conditions.items()]}

  print(f"Final filter: {filter_conditions or 'None'}")

  #--------------------------------------------------------------------------------
  # STEP 3: SIMILARITY SEARCH WITH METADATA FILTER
  #--------------------------------------------------------------------------------

  print(f"\n[3/6] Performing similarity search...")
  print(f"  Query: '{query}'")

  if filter_conditions:
    results = reports_vectorstore.similarity_search(query, k=100, filter=filter_conditions)
  else:
    results = reports_vectorstore.similarity_search(query, k=100)

  print(f"✓ Found {len(results)} relevant chunks")

  #--------------------------------------------------------------------------------
  # STEP 4: FORMAT CONTEXT
  #--------------------------------------------------------------------------------

  print(f"\n[4/6] Formatting context for LLM...")

  context = "\n\n".join([doc.page_content for doc in results])

  print(f"\n✓ Context formatted for LLM...")

  #--------------------------------------------------------------------------------
  # STEP 5: PROMPT TEMPLATE AND COMPOSE CHAIN
  #--------------------------------------------------------------------------------

  print("\n[5/6] Creating prompt template and composing chain...")

  chain = create_rag_chain()

  print("\n✓ Created prompt and Chain composed!")
  #print("\n  Chain structure: \n prompt_template  (formats variables) -> llm (generates response) -> output_parser (extracts string)")

  #--------------------------------------------------------------------------------
  # STEP 6: GENERATE ANSWER BY INVOKING CHAIN
  #--------------------------------------------------------------------------------

  print(f"\n[6/7] Invoking RAG chain with context and query...")

  # Pass variables as dictionary to the chain
  response = chain.invoke({
      "context": context,  # Retrieved reports
      "query": query       # User's question
  }, tools=[{"type": "text"}])

  print(f"\n✓ Answer generated ({len(response)} characters)")

  print("\n" + "="*80)
  print("INFERENCE COMPLETE")
  print("="*80)

  return response  # Returns string response as natural language answer

### Run Inference

In [ ]:
def chat_inference(message, history):
    """
    Gradio ChatInterface wrapper

    Args:
        message (str): Current user message

    Returns:
        str: Bot response
    """

    return inference(message)

In [ ]:
import gradio as gr

#query = "Analisa a GOOGL financeiramente"
#res = inference(query)
#print(res)

demo = gr.ChatInterface(
    fn=chat_inference,
    type="messages",
    title="Stock Analysis RAG",
    description="Ask questions about Google stock.",
    examples=[
        "Summarize the Q2 2024 financial performance of Googl.",
        "Compare the revenue growth of Googl over the past two years.",
        "What are the main risks highlighted in Googl’s 2023 annual report?",
        "Explain the cash flow situation of Googl in Q1 2025.",
        "What trends can be seen in Googl’s R&D expenses?",
        "How did inflation affect Googl in 2024?"
      ],
    )

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b700a35cda2c47da56.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


RUNNING INFERENCE - VERSION v1

[1/6] Detecting report year...
✓ User question year auto-detected: '2024'
✓ User question quarterly period auto-detected: 'unknown'

[2/6] Building metadata filter...
Final filter: {'report_year': {'$eq': '2024'}}

[3/6] Performing similarity search...
  Query: 'How did inflation affect Googl in 2024?'
✓ Found 100 relevant chunks

[4/6] Formatting context for LLM...

✓ Context formatted for LLM...

[5/6] Creating prompt template and composing chain...
✓ Prompt template created

[5/6] Composing chain...

✓ Created prompt and Chain composed!

[6/7] Invoking RAG chain with context and query...

✓ Answer generated (2583 characters)

INFERENCE COMPLETE
RUNNING INFERENCE - VERSION v1

[1/6] Detecting report year...
✓ User question year auto-detected: '2024'
✓ User question quarterly period auto-detected: '2q'

[2/6] Building metadata filter...
Final filter: {'$or': [{'report_year': {'$eq': '2024'}}, {'report_quarter': {'$eq': '2q'}}]}

[3/6] Performing similar